In [262]:
import os
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import pandas as pd
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

In [264]:
# Setup retry strategy
session = requests.Session()
retry = Retry(total=5, backoff_factor=2, status_forcelist=[429, 500, 502, 503, 504])
adapter = HTTPAdapter(max_retries=retry)
session.mount("https://", adapter)

In [265]:
# Extraction layer
def read_data(page=0):
    """
    This function reads data from the collegescorecard api
    page: the current page of the api
    """
    # API parameters
    API_KEY = ""
    params = {
        "api_key": API_KEY,
        "fields": "id,school.name,school.city,school.state,school.ownership,school.school_url,"
                  "school.price_calculator_url,"
                  "latest.admissions.admission_rate.overall,latest.cost.tuition.in_state,latest.cost.tuition.out_of_state,"
                  "latest.earnings.10_yrs_after_entry.median,latest.admissions.sat_scores.average.overall,"
                  "latest.completion.completion_rate_4yr_150nt,latest.student.demographics.student_faculty_ratio,"
                  "school.carnegie_basic,latest.student.demographics.race_ethnicity.non_resident_alien,"
                  "latest.school.instructional_expenditure_per_fte,latest.school.tuition_revenue_per_fte,"
                  "latest.school.endowment.end,latest.student.retention_rate.four_year.full_time_pooled,"
                  "latest.aid.pell_grant_rate,school,school.institutional_characteristics.level,school.carnegie_undergrad",
        "per_page": 100,  
        "page": page
    }
    # Endpoint
    BASE_URL = "https://api.data.gov/ed/collegescorecard/v1/schools"
    try:
        response = session.get(url=BASE_URL, params=params, timeout=30)
        response.raise_for_status()  # Raise HTTP errors
        results = response.json().get('results', [])
        if not results:
            print(f"Warning: Empty response received on page {page}")
            return None
        ownership_mapping = {1: "Public", 2: "Private Nonprofit", 3: "Private For-Profit"}
        research_mapping = {15: 1, 16: 2, 27: 3,17:4, 18: 5, 19: 6, 20: 7} 
        program_type_mapping ={0:'Exclusively Graduate',
                       1:'higher part-time',
                       2:'mixed part/full-time',
                       3:'medium full-time',
                       4:'higher full-time',
                       5:'higher part-time',
                       6:'medium full-time, inclusive, lower transfer-in',
                       7:'medium full-time, inclusive, higher transfer-in',
                       8:'medium full-time, selective, lower transfer-in',
                       9:'medium full-time , selective, higher transfer-in',
                       10:'full-time, inclusive, lower transfer-in',
                       11:'full-time, inclusive, higher transfer-in',
                       12:'full-time, selective, lower transfer-in',
                       13:'full-time, selective, higher transfer-in',
                       14:'full-time, more selective, lower transfer-in',
                       15:'full-time, more selective, higher transfer-in'
                      }
        duration_mapping ={1:'4-year',
                   2:'2-year',
                   3:'less_than_2_years'
                              }
                  
        program_name_mapping={0:'Not classified',
                      1:'Associates Colleges: High Transfer-High Traditional',
                      3: 'Associates Colleges: High Transfer-High Nontraditional',
                      4: 'Associates Colleges: Mixed Transfer/Career & Technical-High Traditional',
                      5:'Associates Colleges: Mixed Transfer/Career & Technical-Mixed Traditional/Nontraditional',
                      6:'Associates Colleges: Mixed Transfer/Career & Technical-High Nontraditional',
                      7:'Associates Colleges: High Career & Technical-High Traditional',
                      8:'Associates Colleges: High Career & Technical-Mixed Traditional/Nontraditional',
                      9:'Associates Colleges: High Career & Technical-High Nontraditional',
                      10:'Health Professions',
                      11:'Technical Professions',
                      12:'Arts & Design',
                      13:'Other Fields',
                      14:'Baccalaureate/Associates Colleges: Associates Dominant',
                      15:'Doctoral Universities: Very High Research Activity',
                      16:'Doctoral Universities: High Research Activity',
                      17:'Doctoral/Professional Universities',
                      18:'Masters Colleges & Universities: Larger Programs',
                      19:'Masters Colleges & Universities: Medium Programs',
                      20:'Masters Colleges & Universities: Small Programs',
                      21:'Baccalaureate Colleges: Arts & Sciences Focus',
                      22:'Baccalaureate Colleges: Diverse Fields',
                      23:'Baccalaureate/Associates Colleges: Mixed Baccalaureate/Associates',
                      24:'Faith-Related Institutions',
                      25:'Medical Schools & Centers',
                      26:'Other Health Professions Schools',
                      27:'Research Institution',
                      28:'Engineering and Other Technology-Related Schools',
                      29:'Business & Management Schools',
                      30:'Arts, Music & Design Schools',
                      31:'Law Schools',
                      32:'Other Special Focus Institutions',
                      33:'Tribal Colleges'
                                }
        data_list =[]
        for result in results:
            data = {
                'SCHOOL_ID': result.get('id'),
                'NAME': result.get('school.name'),
                'CITY': result.get('school.city'),
                'STATE': result.get('school.state'),
                'SCHOOL_TYPE': ownership_mapping.get(result.get('school.ownership'), 'Unknown'),
                'SCHOOL_URL': result.get('school.school_url'),
                'PRICE_CALCULATOR': result.get('school.price_calculator_url'),
                'ADMISSION_RATE': result.get('latest.admissions.admission_rate.overall'),
                'IN_STATE_TUITION': result.get('latest.cost.tuition.in_state'),
                'OUT_OF_STATE_TUITION': result.get('latest.cost.tuition.out_of_state'),
                'AVERAGE_SAT_SCORE': result.get('latest.admissions.sat_scores.average.overall'),
                'EARNINGS_AFTER_10YRS': result.get('latest.earnings.10_yrs_after_entry.median'),
                'GRADUATION_RATE': result.get('latest.completion.completion_rate_4yr_150nt'),
                'faculty_quality': result.get('latest.student.demographics.student_faculty_ratio'),
                'international_outlook': result.get('latest.student.demographics.race_ethnicity.non_resident_alien'),
                'revenue_per_student': result.get('latest.school.tuition_revenue_per_fte'),
                'spending_per_student': result.get('latest.school.instructional_expenditure_per_fte'),
                'endowment': result.get('latest.school.endowment.end'),
                'RETENTION_RATE': result.get('latest.student.retention_rate.four_year.full_time_pooled'),
                'FINANCIAL_AID_PERCENT': result.get('latest.aid.pell_grant_rate'),
                'PROGRAM_NAME':program_name_mapping.get(result.get('school.carnegie_basic'),'Unknown'),
                'duration':duration_mapping.get(result.get('school.institutional_characteristics.level'),'Unknown'),
                'PROGRAM_TYPE':program_type_mapping.get(result.get('school.carnegie_undergrad'),'Unknown'),
                'research_output': research_mapping.get(result.get('school.carnegie_basic'), 8),
                
                }
            data_list.append(data)
        return data_list
    except requests.exceptions.RequestException as e:
        print(f"API Request Error on page {page}: {e}")
        return None

In [266]:
data = []
page= 0

In [267]:
 while True:
        page += 1
        result = read_data(page=page)
        if not result:
            break
        print(f"Currently on page {page}")
        data.extend(result)

Currently on page 1
Currently on page 2
Currently on page 3
Currently on page 4
Currently on page 5
Currently on page 6
Currently on page 7
Currently on page 8
Currently on page 9
Currently on page 10
Currently on page 11
Currently on page 12
Currently on page 13
Currently on page 14
Currently on page 15
Currently on page 16
Currently on page 17
Currently on page 18
Currently on page 19
Currently on page 20
Currently on page 21
Currently on page 22
Currently on page 23
Currently on page 24
Currently on page 25
Currently on page 26
Currently on page 27
Currently on page 28
Currently on page 29
Currently on page 30
Currently on page 31
Currently on page 32
Currently on page 33
Currently on page 34
Currently on page 35
Currently on page 36
Currently on page 37
Currently on page 38
Currently on page 39
Currently on page 40
Currently on page 41
Currently on page 42
Currently on page 43
Currently on page 44
Currently on page 45
Currently on page 46
Currently on page 47
Currently on page 48
C

In [268]:
df = pd.DataFrame(data)

In [270]:
df.head()

,SCHOOL_ID,NAME,CITY,STATE,SCHOOL_TYPE,SCHOOL_URL,PRICE_CALCULATOR,ADMISSION_RATE,IN_STATE_TUITION,OUT_OF_STATE_TUITION,...,international_outlook,revenue_per_student,spending_per_student,endowment,RETENTION_RATE,FINANCIAL_AID_PERCENT,PROGRAM_NAME,duration,PROGRAM_TYPE,research_output
0,105899,Arizona Christian University,Glendale,AZ,Private Nonprofit,arizonachristian.edu/,www.arizonachristian.edu/financial-aid/calcula...,0.7014,31650.0,31650.0,...,0.0321,12924.0,4421.0,419892.0,0.5439,0.3610,Baccalaureate Colleges: Diverse Fields,4-year,"full-time, inclusive, higher transfer-in",8
1,106041,Universal Technical Institute of Arizona Inc,Avondale,AZ,Private For-Profit,www.uti.edu/,www.uti.edu/admissions/tuition,NaN,NaN,NaN,...,0.0000,16466.0,3292.0,NaN,NaN,0.4604,Technical Professions,2-year,higher full-time,8
2,106148,Yavapai College,Prescott,AZ,Public,https://www.yc.edu/,https://www.yc.edu/v6/public/compare/cost-comp...,NaN,2680.0,9340.0,...,0.0025,2099.0,7333.0,16353389.0,NaN,0.1991,Unknown,2-year,higher part-time,8
3,106245,University of Arkansas at Little Rock,Little Rock,AR,Public,ualr.edu/www/,ualr.edu/finaid/npc/npcalc.htm,0.6146,8366.0,18957.0,...,0.0283,5714.0,9365.0,159965311.0,0.6969,0.3971,Doctoral Universities: High Research Activity,4-year,"medium full-time, inclusive, higher transfer-in",2
4,106263,University of Arkansas for Medical Sciences,Little Rock,AR,Public,https://www.uams.edu/,https://studentfinancialservices.uams.edu/tuit...,NaN,NaN,NaN,...,0.0039,16879.0,42864.0,348414506.0,NaN,0.3017,Research Institution,4-year,"medium full-time, inclusive, higher transfer-in",3


# Transformation layer

In [272]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6384 entries, 0 to 6383
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   SCHOOL_ID              6384 non-null   int64  
 1   NAME                   6384 non-null   object 
 2   CITY                   6384 non-null   object 
 3   STATE                  6384 non-null   object 
 4   SCHOOL_TYPE            6384 non-null   object 
 5   SCHOOL_URL             6372 non-null   object 
 6   PRICE_CALCULATOR       5852 non-null   object 
 7   ADMISSION_RATE         1921 non-null   float64
 8   IN_STATE_TUITION       3692 non-null   float64
 9   OUT_OF_STATE_TUITION   3692 non-null   float64
 10  AVERAGE_SAT_SCORE      1069 non-null   float64
 11  EARNINGS_AFTER_10YRS   5260 non-null   float64
 12  GRADUATION_RATE        2237 non-null   float64
 13  faculty_quality        5603 non-null   float64
 14  international_outlook  5617 non-null   float64
 15  reve

In [273]:
df_sorted = df.sort_values(
    by=[
        "research_output","ADMISSION_RATE","GRADUATION_RATE","EARNINGS_AFTER_10YRS","faculty_quality",
        "international_outlook", "revenue_per_student", "spending_per_student",
        "endowment"
    ],
    ascending=[True,True, False,False, True, False, False, False, False],
    na_position='last'
)

In [274]:
df_top_1000=pd.DataFrame(df_sorted.head(1000))

In [277]:
columns=['PRICE_CALCULATOR','IN_STATE_TUITION',
         'OUT_OF_STATE_TUITION','AVERAGE_SAT_SCORE','EARNINGS_AFTER_10YRS',
         'GRADUATION_RATE','international_outlook','revenue_per_student',
         'spending_per_student','endowment','RETENTION_RATE','FINANCIAL_AID_PERCENT','faculty_quality']

In [280]:
df_top_1000[columns]=df_top_1000[columns].fillna(0)

In [281]:
df_top_1000['ADMISSION_RATE'].fillna(1,inplace=True)

In [282]:
df_top_1000["PRICE_CALCULATOR"] = df_top_1000["PRICE_CALCULATOR"].astype(str)

# Loading layer

In [283]:
conn = snowflake.connector.connect(
    user='CHOICEUGWUEDE',
    password='3sUEi4bqnQyehzB',
    account='KVPAJQA-ZQ87651',
    warehouse='DEC_WH',
    database='UNIVERSITIES_DB',
    schema='PUBLIC'
)

In [284]:
cur = conn.cursor()

In [285]:
# EXTRACT UNIQUE PROGRAMS & PROGRAM TYPES 
dim_program = df_top_1000[['PROGRAM_NAME']].dropna().drop_duplicates().reset_index(drop=True)
dim_program['PROGRAM_ID'] = range(1, len(dim_program) + 1)

dim_program_type = df_top_1000[['PROGRAM_TYPE']].dropna().drop_duplicates().reset_index(drop=True)
dim_program_type['PROGRAM_TYPE_ID'] = range(1, len(dim_program_type) + 1)

In [286]:
# CREATE DIMENSION TABLES 
cur.execute("""
    CREATE TABLE IF NOT EXISTS dim_program (
        program_id INT AUTOINCREMENT PRIMARY KEY,
        program_name VARCHAR(255) UNIQUE
    )
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS dim_program_type (
        program_type_id INT AUTOINCREMENT PRIMARY KEY,
        program_type VARCHAR(255) UNIQUE
    )
""")
print("dimension tables created")

dimension tables created


In [287]:
# Load DataFrame into a Snowflake temporary table and upsert into main table 
def upsert_dimension_table(table_name, df, key_column, value_column):
    temp_table = f"staging_{table_name}"
    
    # Create Staging Table
    cur.execute(f"CREATE OR REPLACE TEMP TABLE {temp_table} AS SELECT * FROM {table_name} WHERE 1=0")

    # Load Data into Staging Table
    for _, row in df.iterrows():
        cur.execute(f"""
            INSERT INTO {temp_table} ({value_column})
            SELECT '{row[value_column]}' WHERE NOT EXISTS (
                SELECT 1 FROM {table_name} WHERE {value_column} = '{row[value_column]}'
            )
        """)

    # Merge Data from Staging to Main Table
    cur.execute(f"""
        MERGE INTO {table_name} AS target
        USING {temp_table} AS source
        ON target.{value_column} = source.{value_column}
        WHEN NOT MATCHED THEN INSERT ({value_column}) VALUES (source.{value_column});
    """)



In [289]:
# Perform Upserts
upsert_dimension_table('dim_program', dim_program, 'program_id', 'PROGRAM_NAME')
upsert_dimension_table('dim_program_type', dim_program_type, 'program_type_id', 'PROGRAM_TYPE')
print("Dimension tables upsert successful!")

Dimension tables upsert successful!


In [290]:
# MAP PROGRAM & PROGRAM TYPE IDs BACK TO OUR DF
df_top_1000 = df_top_1000.merge(dim_program, on='PROGRAM_NAME', how='left')
df_top_1000 = df_top_1000.merge(dim_program_type, on='PROGRAM_TYPE', how='left')

## University fact table

In [291]:
#Create staging table 
cur.execute("CREATE OR REPLACE TEMP TABLE staging_university AS SELECT * FROM university WHERE 1=0")

In [292]:
df_top_1000 = df_top_1000.reset_index(drop=True)

In [293]:
columns=['SCHOOL_ID','NAME','CITY','STATE','SCHOOL_URL','ADMISSION_RATE','PRICE_CALCULATOR','FINANCIAL_AID_PERCENT',
         'IN_STATE_TUITION','OUT_OF_STATE_TUITION','RETENTION_RATE','GRADUATION_RATE','AVERAGE_SAT_SCORE',
         'EARNINGS_AFTER_10YRS','SCHOOL_TYPE','PROGRAM_ID','PROGRAM_TYPE_ID'
]

In [294]:
df_university=df_top_1000[columns]

In [295]:
#conn.cursor().execute("TRUNCATE TABLE STAGING_UNIVERSITY")

In [296]:
success, row_count, total_rows, load_details = write_pandas(conn, df_university, "STAGING_UNIVERSITY")
if success:
    print("Data successfully loaded into staging_university!")

Data successfully loaded into staging_university!


In [297]:
update_program_id_query = """
MERGE INTO staging_university su
USING dim_program p
ON su.program_id = p.program_id
WHEN MATCHED THEN
    UPDATE SET su.program_id = p.program_id;
"""
cur.execute(update_program_id_query)
conn.commit()
print("Updated program_id successfully in staging_university.")


Updated program_id successfully in staging_university.


In [298]:
update_program_type_id_query = """
MERGE INTO staging_university su
USING dim_program_type pt
ON su.program_type_id = pt.program_type_id
WHEN MATCHED THEN
    UPDATE SET su.program_type_id = pt.program_type_id;
"""
cur.execute(update_program_type_id_query)
conn.commit()
print("Updated program_type_id successfully in staging_university.")


Updated program_type_id successfully in staging_university.


In [300]:
# Merge data from staging_university into university
merge_query = """
MERGE INTO university AS u
USING staging_university AS s
ON u.school_id = s.school_id  
WHEN MATCHED THEN 
    UPDATE SET 
        u.name = s.name,
        u.city = s.city,
        u.state = s.state,
        u.school_url = s.school_url,
        u.admission_rate = s.admission_rate,  
        u.price_calculator = s.price_calculator,
        u.in_state_tuition = s.in_state_tuition,
        u.out_of_state_tuition = s.out_of_state_tuition,
        u.average_sat_score = s.average_sat_score,
        u.earnings_after_10yrs = s.earnings_after_10yrs,
        u.graduation_rate = s.graduation_rate,
        u.retention_rate = s.retention_rate,
        u.financial_aid_percent = s.financial_aid_percent,
        u.program_id = s.program_id,  
        u.program_type_id = s.program_type_id,  
        u.school_type = s.school_type
WHEN NOT MATCHED THEN
    INSERT (
        school_id, name, city, state, school_url,
        price_calculator, admission_rate,
        in_state_tuition, out_of_state_tuition, average_sat_score,
        earnings_after_10yrs, graduation_rate,
        retention_rate, financial_aid_percent,
        program_id, program_type_id, school_type
    ) VALUES (
        s.school_id, s.name, s.city, s.state, s.school_url,
        s.price_calculator, s.admission_rate,
        s.in_state_tuition, s.out_of_state_tuition, s.average_sat_score,
        s.earnings_after_10yrs, s.graduation_rate,
        s.retention_rate, s.financial_aid_percent,
        s.program_id, s.program_type_id, s.school_type
    );
"""

cur.execute(merge_query)
conn.commit()  

print("Update and merge operations completed successfully!")


Update and merge operations completed successfully!
